# 검색과 응답을 최적화하는 RAG 고도화 전략
- RAG 고도화는 관련 문서를 가져오는 Retrieval 파트와 응답을 생성하는 Generation 파트 모두에서 달성할 수 있습니다.  

<table>
    <thead>
        <tr>
            <th>구분</th>
            <th>주요 개선 요소</th>
            <th>개략적 개선 방향</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan="4">Retrieval<br />(문서 검색)</td>
            <td>청킹 전략<br />(전처리)</td>
            <td>문서를 효과적으로 분할하는 다양한 기법을 활용하여 검색에 최적화딘 형태로 변환함으로써 검색 품질과 처리 속도 향상</td>
        </tr>
        <tr>            
            <td>질의 변형<br />(쿼리 재작성)</td>
            <td>질문의 구체화, 가상의 답변 활용 등의 기법을 통해 사용자의 원래 질문을 검색에 효율적인 형태로 재구성</td>
        </tr>        
        <tr>            
            <td>검색 알고리즘<br />최적화</td>
            <td>의미 기반의 벡터 검색과 키워드 기반 검색, 하이브리드 검색 등의 방식을 활용하여 관련성 높은 문서를 더 효율적으로 검색</td>
        </tr>           
        <tr>            
            <td>문서 후처리<br />(리랭킹)</td>
            <td>1차 검색 결과와 질문의 연관성을 재평가하여 질문과 관련없는 문서 제거, 이를 통해 검색된 정보의 품질을 높이고 궁극적으로 더 신뢰할 수 있는 최종 프롬프트 제공</td>
        </tr>
        <tr>
            <td rowspan="2">Generation<br />(답변 생성)</td>
            <td>Self-RAG</td>
            <td>모델이 스스로 추가 검색 필요성을 판단하고 부족한 정보를 찾아 보완. 이를 바탕으로 더 깊이 있고 정확한 응답을 제공하여 사용자 경험 개선</td>
        </tr>                                
        <tr>            
            <td>모델 파인튜닝</td>
            <td>모델을 학습시켜 다양한 상황에 대한 대응 능력 강화. 이를 통해 검색된 정보를 더욱 효율적으로 활용하여 맥락에 맞는 적절한 응답 생성 가능</td>
        </tr>        
    </tbody>    
</table>

## 1. 청킹 전략
- 문서 전처리 단계는 RAG 시스템에서 실제 검색에 사용될 문서들을 가공하여, 검색과 생성에 최적화된 형태로 변환하는 과정입니다.
- 문서 분할은 긴 문서를 더 작고 관리하기 쉬운 단위로 나누는 과정입니다.
- 효과적인 분할 방식은 관련 정보의 정확도를 높이고, 문맥의 일관성을 유지하는 데 도움을 줍니다.
- 가장 기본적인 분할 방식은 문자 수 기반 분할입니다.
- 구현히 가장 간단하고 빠르지만, 의미적 구조를 고려하지 않기 때문에, 중요한 정보가 분할되어 검색시 누락되거나 문맥이 왜곡될수 있습니다.

- 재귀적 문자 텍스트 분할 방식은 문서의 구조를 고려하여 분할하지만, 의미를 고려하지 않고 기계적인 규칙에 따라 텍스트를 분할하기 때문에 연관된 내용이 분리되어 문맥이 끊길 수 있는 문제가 있습니다.
- 의미 기반 분할은 텍스트의 의미적 연관성을 고려햐여 일관된 청크를 생성하는데 효과적이지만, 긴 문서나 복잡한 주제를 다룰 때 문서의 전체적인 구조와 계층적 관계를 완벽하게 포착하기에는 한계가 있습니다.

[ch04_PARENT_CHILD_CHUNKING.ipynb](ch04_PARENT_CHILD_CHUNKING.ipynb)

## 2. 질의 변형
- 사용자의 원래 질문을 보다 효과적인 검색 쿼리로 변환하는 과정을 말합니다.
- 사용자의 질문이 모호하거나 검색에 최적화되지 않은 형태인 경우, 연관 문서를 제대로 검색할수 없어 RAG 시스템의 정확도가 떨어지게 됩니다.
- 질의 변형은 원래의 질문을 분석하고 재구성하여 이러한 문제를 해결합니다.

### 2.1 다중 질의 생성
- 사용자의 원래 질문을 바탕으로 여러 개의 다양한 쿼리를 생성하는 방법입니다.
- LLM을 사용하여 원래 질문을 다양한 관점으로 변형시킨 여러 개의 질문을 생성합니다.
- 생성된 각 쿼리를 사용하여 독립적으로 문서를 검색합니다.(병렬검색)
- 검색 결과를 종합하여 중복을 제거하고 가장 관련성 높은 문서들을 선별합니다.

##### 예시
- 주식 투자를 처믕 시작하려면 어떻게 해야 하나요?
- 초보 투자자를 위한 주식 투자 기초 지식은 무엇인가?
- 주식 시장 분석을 위한 기본적인 재무제표 읽는 법은?
- 주식 투자 시작 전 필요한 자금 관리 전략은?

- 보다 다양한 측면에서 정보를 포괄적으로 검색하여 다양한 문서를 수집하게 합니다.
- 하나의 질문을 여러 가지 방식으로 해석하고 쿼리를 생성함으로써, 사용자의 의도를 더 정확히 파악하고 그에 맞는 결과를 제공합니다.

[ch04_MULTIQUERY_GENERATION.ipynb](ch04_MULTIQUERY_GENERATION.ipynb)

## 2.2 가상 문서 임베딩(HyDE)
- 기존 쿼리에 대해 가상의 문서를 만들어서 이를 기존 쿼리 대신 활용하는 방법입니다.
- 질문을 이용하는 것이 아닌, 질문에 대한 답변을 기반으로 문서를 찾는 방식입니다.
- 검색 대상 문서가 질문보다 답변과 더 높은 의미적 연관성을 가지는 경우가 많다는 점을 활용한 접근입니다.

- 예시
- 금융 서적을 대상으로 한 RAG 시스템을 예로 생각해 봅시다.
- 편의상 이 시스템의 유사도 검색은 키워드를 활용한 bm25 방식을 활용한다고 가정합니다.
- "주식시장의 변동성이 높을 때 투자 전략은 무엇인가요?"와 같은 질문을 그대로 검색에 활용한다고 가정할때 사용할 수 있는 키워드는 다음과 같습니다.
  - 주식시장, 변동성, 투자 전략
- 이 질문에 대한 답변인 "주식 시장의 변동성이 높을 때는 분산 투자, 달러 코스트 애퍼리징, 안전 자산 비중 확대 등의 전략을 고려할 수 있습니다."
  - 분산 투자, 달러 코스트 애버리징, 안전 자산, 비중 확대
- 답변을 활용하는 방식으로 문서 검색 시 더 정확하고 관련성 높은 문서를 찾을 가능성을 크게 높입니다.
- 단점은 가상의 답변을 생성하기 위한 LLM을 사용하기 때문에 계산 비용이 발생합니다.
- 검색 지연시간이 증가할수 있어 빠른 응답이 요구되는 애플리케이션에서는 제약이 될 수 있습니다.

[ch04_HYDE.ipynb](ch04_HYDE.ipynb)

# 3. 검색 알고리즘
- 검색 알고리즘은 사용자가 작성한 쿼리와 참조 문서 간의 관련성을 평가하고 가장 적합한 문서를 선별하는 로직을 뜻합니다.
- 희소검색(sparse retrieval)
  - 희소 검색은 전통적인 키워드 기반 방식으로, 대표적인 예로 BM25 알고리즘이 있습니다.
  - 문서와 쿼리 간에 일치하는 키워드 중심으로 관련성을 평가합니다.
  - 구현이 간단하고 계산 효율성이 높지만, 의미적 유사성을 포착하는 데 한계가 있을 수 있습니다.
- 밀집검색(dense retrieval)
  - 밀집 검색은 쿼리와 문서를 고차원 벡터 공간에 임베딩하여 비교하는 방식입니다.
  - 단어의 의미와 문맥을 포착할 수 있는 신경망 기반 임베딩 모델을 사용하여 쿼리와 문서를 벡터화하고, 이들 간의 유사도를 계산하여 관련성을 판단합니다.
  - 계산 비용이 크고, 대규모 데이터셋을 학습하는 과정이 필요하기 때문에 리소스 측면에서 부담이 될수 있습니다.

## 3.1 희소 검색
- 문서와 쿼리를 희소 벡터 형태로 표현하여 검색을 수행하는 방법입니다.
- 희소 벡터는 전체 어휘 사전의 크기에 해당하는 차원을 가진 벡터로, 해당 문서나 쿼리에 등장하는 단어에 해당하는 위치만 1(또는 다른 가중치 값)이고 나머지는 모두 0인 형태를 갖습니다.


### 희소 벡터의 예
- 금융 뉴스 검색 시스템의 어휘 사전이 500,000개의 단어로 구성되어 있다고 가정해 보겠습니다.
- "Tesla stock surges as electric vehicle demand rises"
- 0,...,1(Tesla),...,0, ..., 1(stock),...
- 이 벡터는 500,000개의 요소를 가지며, 그중 8개의 요소만 1이고 나머지 499,992개의 요소는 0입니다.
- 이처럼 벡터의 대부분이 0으로 채워져 있어 듬성듬성하다는 의미로 희소 벡터라고 부릅니다.

### 희속 검색의 단계별 동작 과정
1. 문서 변환: 각 단어의 출현 정보를 저장하는 방식으로 모든 문서를 희소 벡터 기반으로 변환합니다. 한국어의 경우, 이 과정에서 불용어 제거, 어간 추출 등의 전처리가 수행될 수 있습니다.
2. 질문 처리: 사용자의 질문을 동일한 방식으로 벡터화합니다.
3. 유사도 계산: 질문 벡터와 문서 벡터 간의 유사도를 계산합니다. 흔히 사용하는 방법으로는 코사인 유사도가 있습니다.
4. 랭킹: 계산된 유사도를 기반으로 문서들의 순위를 매깁니다.
5. 결과 반환: 가장 유사도가 높은 상위 N개의 문서를 결과로 반환합니다.

### 희소 검색의 대표적인 방식
#### TF-IDF(Term Frequency-Inverse Document Frequency)
- TF-IDF는 단어의 중요도를 문서 내 빈도와 전체 문서에서 희소성을 고려하여 관련성을 계산하는 방식입니다.
- 이 방법은 다음 두 요소를 곱하여 각 단어의 최종 TF-IDF 점수를 계산합니다.
  - TF (Term Frequency)
    - 특정 단어가 한 문서에서 얼마나 자주 등장하는지를 나타냅니다. 빈도가 높을수록 해당 문서의 주제와 관련이 깊다고 판단합니다.
    - 계산방법: TF(t, d) = (단어 t의 문서 d 내의 등장횟수) / (문서 d의 총 단어 수)
  - IDF (Inverse Document Frequency)
    - 특정 단어가 전체 문서 집합에서 얼마나 희귀한지를 나타냅니다. 많은 문서에 공통적으로 나타나는 단어(예: 그리고, 그러나)는 덜 중요하다고 판단하여 가중치를 낮춥니다.
    - 소수의 문서에만 나타나는 희귀한 단어일수록 높은 가중치를 받습니다.
    - 계산방법: IDF(t, D) = log(전체 문서 수/ 단어 t가 등장하는 문서 수)
  - TF-IDF 계산식
    - TF-IDF(t, d, D) = TF(t,d) * IDF(t, D)
  - 여기서 t는 단어, d는 개별 문서, D는 전체 문서 집합을 나타냅니다.
  - 결론적으로 TF-IDF는 문서 내에서 자주 등장하면서도 전체 문서 집합에서는 비교적 드물게 나타나는 단어에 높은 점수를 부여합니다.
  - 개별 문서의 특징을 잘 나타내는 핵심 단어를 식별할 수 있으며, 불용어와 같이 빈번하지만 의미적으로 중요하지 않은 단어들의 영향을 줄일 수 있다는 장점이 있습니다.
  - 단점
    - 문서 길이 고려 부족: 긴 문서에서는 단어 빈도가 자연스럽게 높아질 수 있어, 문서 길이에 따른 편향이 발생할 수 잇는 문제점이 있습니다.
    - 단어 빈도의 선형적 증가: 단어 빈도가 증가함에 따라 TF 값이 계속 선형적으로 증가하여, 과도하게 반복되는 단어에 지나치게 높은 가중치를 매기는 일이 발생합니다.

#### BM25(Best Matching 25)
- BM25 알고리즘은 TF-IDF의 기본 아이디어를 유지하면서 문서 길이와 단어 빈도의 영향을 보다 정교하게 조정하여 검색 정확도를 높입니다.
  - 문서 길이 정규화: 문서의 길이를 고려하여 긴 문서에서 단어 빈도를 적절히 조정합니다.
  - 단어 빈도의 포화 처리: 단어 빈도가 증가함에 따라 점수 증가율이 감소하도록 설계되어 과도한 반복이 결과에 미치는 영향을 제한합니다.
- 계산식
# $\text{score}(D, Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot \left(1 - b + b \cdot \frac{|D|}{\text{avgdl}}\right)}$
- D: 문서
- Q: 쿼리(여기서 쿼리는 사용자가 정보 검색 시스템에 전달하는 하나 이상의 키워드, 검색어 또는 문구를 말함)
- f(qi, D): 문서D에서 쿼리 단어 qi의 빈도수
- |D|: 문서의 길이
- avgdl: 전체 문서 집합의 평균 문서 길이
- k1, b: 알고리즘 조정 매개변수(일반적으로 k1은 1.2~2.0, b는 0.75)

  



- BM25에도 IDF가 계산식에 포함되어 있습니다.
- 계산식 마지막 부분에 (1-b+b*|D| / avgdl)을 추가하여 문서 길이에 따른 편향을 줄이도록 만들었습니다.
- 계산식의 f(qi, D) * (k1 + 1) / f(qi, D) + k1 * ... 부분은 f(qi, D)가 증가함에 따라 점수 증가율이 감소하는 형태가 되도록 합니다.
- 이를 통해 단어 빈도가 높아졌을 때에도 점수가 지나치게 높아지지 않도록 조절합니다.

### 형태소 전처리를 통한 BM25 성능 향상
- 한국어와 같은 교착어에서는 영어와 달리 동일한 의미의 단어도 매우 다양한 형태로 활용됩니다.
- 형태소 분석을 통해 전처리를 수행하는 것이 검색 성능 향상에 큰 영향을 미칩니다.

1. 기본적인 형태소 분석
- 문장을 최소 의미 단위인 형태소로 분리하고, 각 형태소에 품사 태그를 부여하는 과정입니다.
  - 입력: "먹었다"
  - 분석 결과: "먹/VV 었/EP + 다/EF"
- 이를 통해 동사 "먹다"(VV)의 기본형, 과거 시제 선언말어미 "었"(EP), 종결어미 "다"(EF)를 파악할 수 있습니다.
- 문장의 구조와 각 요소의 문법적 기능을 정확히 이해할 수 있으며, 핵심 의미를 가진 형태소를 효과적으로 추출할 수 있습니다.
2. 주요 품사 추출
- 형태소 분석 후 명사(NNG, NNP), 동사(VV), 형용사(VA)등 핵심 의미를 전달하는 품사만을 선택적으로 추출합니다.
  - 입력: "아름다운 꽃이 피었습니다"
  - 추출 결과: "아름답/VA", "꽃/NNG", "피/VV"
- 조사나 어미 등 검색에 큰 영향을 미치지 않는 요소를 제거하여 노이즈를 줄일 수 있습니다.
- 핵심 키워드에 집중함으로써 검색 쿼리와 문서 간의 매칭 정확도를 높일 수 있습니다.
3. 복합명사 처리
- 한국어에서는 여러 명사가 결합하여 복합명사를 이루는 경우가 많습니다. 복합명사를 구성요소로 분리하여 처리하면 검색 성능을 향상시킬 수 있습니다.
  - 입력: "정보검색시스템"
  - 분리 결과: "정보/NNG + 검색/NNG + 시스템/NNG"
- 복합명사를 분리함으로써 각 구성요소에 대한 개별 검색이 가능해지며 부분 일치 검색의 정확도를 높일 수 있습니다.
4. 어근 추출 및 표제어 처리
- 동사와 형용사의 다양한 활용 형태를 기본형(표제어)으로 통일하여 처리합니다.
  - "먹었습니다." -> "먹다"
  - "먹고 있다" -> "먹다"
  - "먹을 것이다" -> "먹다"
- 어근을 추출하면 단어의 일관성이 유지되어 검색 매칭 확률이 높아집니다.
- 전처리 과정을 거치면 쿼리에서 실질적 의미를 가진 키워드만 추출할 수 있고, 단어 형태가 달라도 동일한 의미로 처리할 수 있어 검색 성능이 향상됩니다.

[ch04_BM25.ipynb](ch04_BM25.ipynb)

## 3.2 밀집 검색
- 밀집 검색(Dense Retrieval)은 문서와 쿼리를 고차원의 밀집 벡터 형태로 표현하여 검색을 수행하는 방법입니다.
- 단어의 존재여부나 빈도만이 아닌 단어의 의미와 문맥을 고려하여 검색을 수행합니다.
- 이 방식의 핵심은 트랜스포머 기반의 임베딩 모델을 사용하여 텍스트를 의미 공간에 매핑하는 것입니다.
- 텍스트의 의미라는 정성적 지표를 수치화된 벡터로 변환하여 정량적으로 표현하고 계산할 수 있게 만드는 것입니다.
- 텍스트 간의 의미적 유사성을 수학적으로 측정할 수 있게 되어 더 정확하고 맥락을 고려한 검색을 수행할 수 있습니다.

### 밀집 벡터의 예
- 금융 뉴스 검색 시스템에서 다음과 같은 주식 관련 뉴스 헤드라인이 있다고 가정해 봅시다
- "테슬라 주가, 전기차 수요 증가로 급등"
- 이 헤드라인의 밀집 벡터 표현은 다음과 같을 수 있습니다.(300차원의 벡터를 가정)
- [0.25, -0.10, 0.45, ..., 0.05]
- 이 벡터는 헤드라인의 의미를 정량적으로 표현한 수치 데이터입니다.
- 각 차원은 개별적으로 특정 의미를 갖지 않지만, 모든 차원이 함께 결합되어 텍스트의 전체적인 의미를 고차원 공간에 나타냅니다.
- 이를 통해 정성적이었던 텍스트의 의미를 정량적으로 활용할 수 있게 됩니다.
- 의미적 유사성 계산: 다른 뉴스 헤드라인과의 벡터 거리를 계산하여, 얼마나 비슷한 내용을 담고 있는지 수치적으로 평가할 수 있습니다.
- 데이터 분석 및 시각화: 텍스트 데이터를 수치화함으로써 클러스터링이나 분류와 같은 기계학습 알고리즘에 적용할 수 있으며, 이를 통해 패턴이나 트렌드를 발견할 수 있습니다.

- 밀집 벡터 기반 검색법은 의미 기반 검색 방식이므로 맥락을 고려하여 보다 정확한 검색을 가능하게 합니다.
- 동시에 높은 계산 복잡도를 요구한다는 단점이 있습니다.
- 새로운 도메인에 적용할 때 임베딩 모델의 파인튜닝이 필요한 경우도 있습니다.

1. 문서 변환: 모든 문서를 사전 학습된 트랜스포머 기반 임베딩 모델을 사용하여 고차원의 밀집 벡터로 변환하고 저장합니다.
2. 질문 처리: 사용자의 질문 역시 동일한 임베딩 모델을 사용하여 밀집 벡터로 변환합니다.
3. 유사도 계산: 질문 벡터와 문서 벡터 간의 유사도를 계산합니다. 주로 코사인 유사도가 사용됩니다.
4. 랭킹: 계산된 유사도를 기반으로 문서들의 순위를 매깁니다.
5. 결과 반환: 가장 유사도가 높은 상위 N개의 문서를 결과로 반환합니다.

- 밀집 검색은 희소 검색에 비해 계산 복잡도가 높기 때문에, 대규모 시스템에서는 계산 시간이 중요한 문제가 됩니다.
- 예를 들어, 수백만 개의 문서를 다루는 검색 엔진을 생각해봅시다.
- 각 문서가 300차원의 벡터로 표현된다고 할 때, 새로운 검색 쿼리가 들어올 때마다 이를 수백만 개의 벡터와 일일이 비교하면 막대한 시간과 자원이 소모됩니다.
- 이러한 문제를 해결하기 위해 여러 유사도 검색 방식이 등장했습니다. 그중 많이 쓰이는 방법이 FAISS(Facebook AI Similarity Search) 입니다.

[ch04_DENSE_RETRIEVAL.ipynb](ch04_DENSE_RETRIEVAL.ipynb)

## 3.3 앙상블 검색

[ch04_ENSEMBLE_RETRIEVAL.ipynb](ch04_ENSEMBLE_RETRIEVAL.ipynb)

# 4 문서 후처리
- 문서 후처리는 초기 검색 알고리즘으로 검색된 문서들을 대상으로, 보다 정교한 방법을 사용하여 문서의 관련성을 재평가하고 순위를 재조정하는 과정입니다.
- 이 과정에서 핵심적인 역할을 하는 것이 리랭킹입니다. 리랭킹을 통해 초기 검색 알고리즘의 한계를 보완하고 검색 결과의 정확도를 높일 수 있습니다.
- 희소 검색, 밀집 검색등의 초기 검색 알고리즘들은 대규모 문서 집합에서 빠르게 관련 문서를 찾아내는 데 효과적입니다.
- 하지만 기계적인 방식에 의존하기 때문에 문서의 관련성을 완벽하게 평가하는 데 한계가 있습니다.
- 대규모 언어 모델은 더 높은 정확도를 제공할 수 있지만, 대규모 문서 집합 관련성을 모두 파악하기에는 계산 비용이 높고 시간이 오래 걸린다는 단점이 있습니다.

- 초기 검색 방법과 리랭커를 순차적으로 적용하는 전략이 있습니다.
- 초기 검색을 통해 빠르게 문서를 찾아냅니다. 속도는 빠르지만 관련성이 떨어지는 문서도 함께 선별될 수 있습니다.


"주식 시장 투자 전략"
- 희소 검색 결과: 현대 주식 투자 전략, 주식 시장 역사
- 밀집 검색 결과: 증권 분석 방법, 금융 시장 예측 기법


- 트랜스포머 기반 리랭커는 단순한 키워드 매칭이나 의미적 유사성을 넘어 전체적인 맥락과 잠재적 가치를 평가할 수 있습니다.
- 관련성 점수 : 해당 문서가 주제와 관련 있을 가능성을 상대적으로 나타내는 척도입니다.

- 현대 주식 투자 전략: 3.5
- 주식 시장 역사: 1.2
- 증권 분석 방법: 2.8
- 금융 시장 예측 기법: 2.1

- 리랭커는 이 값들을 바탕으로 관련성 점수 값이 높은 순으로 재정렬하여, 상위 n개를 선택하는 방식으로 최종 선별을 수행합니다.

- 현대 주식 투자 전략: 3.5
- 증권 분석 방법: 2.8

- 이러한 초기 검색과 리랭킹의 두 단계 접근 방식을 통해, 방대한 양의 자료 중에서 빠르게 관련 있는 자료를 찾아내고, 가장 적절한 자료를 정확하게 선별할 수 있습니다.

## 4.1 고성능 대규모 언어 모델 기반 리랭킹

[ch04_RERANKER_LLM.ipynb](ch04_RERANKER_LLM.ipynb)

## 4.2 크로스 인코더 기반 리랭킹
[ch04_RERANKER_CROSS_ENCODER.ipynb](ch04_RERANKER_CROSS_ENCODER.ipynb)

# 5. 확장된 RAG 방법론
## 5.1 Self-RAG 개요
- 청킹 전략, 질의 변형, 검색 알고리즘, 리랭킹 등의 기술들로 RAG 시스템의 성능을 크게 향상시킬수 있습니다.
- 전통적인 RAG 시스템은 여전히 몇 가지 구조적 한계를 지니고 있습니다.
- 사용자의 질문이 문서와 관련이 없더라도 정해진 수의 문서를 무작위로 검색하고 이를 프롬프트에 통합하는 방식은 부정확하거나 부적절한 응답을 초래할 수 있습니다.
- 검색된 정보를 효과적으로 활용하지 못하거나 원래 질문의 맥락을 잃어버리는 경우도 발생합니다.


- Self-RAG는 LLM이 기존 RAG의 각 단계에 직접 개입하여 보다 정교한 제어를 수행하는 방식입니다.
- LLM은 각 단계에서 특수 토큰을 출력하여 검색의 필요 여부를 판단하고, 검색한 정보를 평가하여 최종 응답 생성 과정을 관리합니다.
- LLM은 상황에 따라 필요한 경우에만 문서를 검색하고, 정보를 스스로 검증한 뒤, 보다 신뢰성 높은 답변을 생성할 수 있습니다.

### Self-RAG의 작동방식
- 검색, 생성, 평가의 3가지 주요단계로 작동합니다.
- 각 단계에서 LLM이 직접 특수 토큰을 출력하여 프로세스를 제어하고 결과를 개선합니다.
- 모델은 검색이 필요한지 여부를 스스로 판단하고, 검색된 정보를 평가하며, 답변을 생성하고 검토하는 전 과정을 주도합니다.

"테슬라 주식이 향후 1년 동안 좋은 투자처가 될까요?"

#### 1. 검색
- Self-RAG는 먼저 사용자의 질문을 분석하여 외부 데이터 검색이 필요한지 결정합니다.
- 이 과정에서 Retrieve 토큰을 활용합니다.
- Retrieve=Yes: 외부 데이터가 필요한 경우
- Retrieve=No: 외부 데이터가 필요하지 않은 경우

- 예시 질문은 최신 정보와 경제적 지표를 포함해야 합니다. 모델은 외부 데이터를 검색하는 것이 필요하다고 판단할 가능성이 큽니다.
- Retrieve=Yes 토큰을 출력하고, 테슬라의 최근 주간 동향, 분석가 평가, 경제 보고서 등의 관련 정보를 검색합니다.

#### 2. 생성
- 검색된 정보가 있으면, Self-RAG는 이를 바탕으로 답변을 생성합니다.
1. 관련성 평가
  - Self-RAG는 검색된 문서의 관련성을 평가합니다. 이 과정에서는 ISREL 토큰을 사용하여 관련성을 나타냅니다.
  - ISREL=Relevant: 문서가 질문과 관련 있음
  - ISREL=Irrelevant: 문서가 질문과 관련 없음
2. 답변 생성
  - 관련 있는 문서가 존재한다면 이를 바탕으로 사용자 질문에 대한 답변을 생성합니다.
  - 이 과정에서 모델은 자신의 기존 지식과 검색된 정보를 결합하여 포괄적인 답변을 만듭니다.
  - 관련 정보가 있는 문서가 없다면, 검색된 문서를 무시하고 모델의 기존 지식을 활용하여 답변합니다.

#### 3. 평가
- Self-RAG는 생성된 답변의 유용성을 평가하고, 필요시 추가 검색이나 수정을 통해 답변을 개선합니다.
1. 지원 평가
  - 생성된 답변의 각 부분이 검색된 정보로 얼마나 뒷받침되는지 평가합니다. ISSUP토큰을 사용합니다.
  - ISSUP=Fully supported: 완전히 지원됨
  - ISSUP=Partially supported: 부분적으로 지원됨
  - ISSUP=No support: 지원 없음
2. 유용성 평가
  - 생성된 답변이 사용자 질문에 얼마나 유용한지 평가합니다. 이때 ISUSE 토큰을 사용합니다.
  - ISUSE=1 (최저) ~ ISUSE=5 (최고)

#### 4. 최종 출력
- 모든 평가와 필요한 개선 과정을 거친 후, Self-RAG는 최종 답변을 출력합니다.
- 생성된 텍스트 답변
- 사용된 정보 출처에 대한 인용
- 각 세그먼트에 대한 평가 결과(ISREL, ISSUP, ISUSE 토큰 값)

- 요약하자면, Self-RAG 방식의 핵심은 자체 반영으로 볼 수 있습니다.
- 각 단계마다 LLM이 자신의 출력을 평가하고 스스로 개선하는 과정을 거칩니다.
- 이러한 과정을 통해 Self-RAG는 더 정확하고 유용한 답변을 생성할 수 있습니다.